Import the required libraries ↙

In [2]:
pip install  validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=061b2ce34410b9f99f5a272e05f5db0e634cf1c12963009b32ed488e8ecf986e
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [3]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.0
    Uninstalling Werkzeug-2.3.0:
      Successfully uninstalled Werkzeug-2.3.0


In [5]:
pip install dash-daq


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669696 sha256=53622eac0fb9a71440be2fc8fe328c0e6bc3d6d9571fcbac54876d8932a0b182
  Stored in directory: /root/.cache/pip/wheels/75/14/1b/208d09d5e239391048bdc167759977b41ba65a3d4063aebf6b
Successfully built dash-daq


In [6]:
pip install dash-bootstrap-components


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 5.5 MB/s eta 0:00:00


In [7]:
pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import validators
import random
import numpy as np
import dash
import dash_core_components as dcc
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
import re
import ipywidgets as widgets
from IPython.display import display
import base64
from dash import Dash, callback_context, html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash_daq as daq
import pandas as pd
import requests
from io import BytesIO
from PIL import Image, UnidentifiedImageError


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning:

Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning

<ipython-input-8-5238a85aee9e>:14: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`



#Load Datasets  ⏳

In [9]:

links = "/content/sample_data/dataset/links.csv"
movies = "/content/sample_data/dataset/movies.csv"
ratings = "/content/sample_data/dataset/ratings.csv"
tags = "/content/sample_data/dataset/tags.csv"
images="/content/sample_data/dataset/images.csv"


links = pd.read_csv(links)
movies = pd.read_csv(movies)
ratings = pd.read_csv(ratings)
tags = pd.read_csv(tags)
images=pd.read_csv(images, encoding='latin-1')


Merge the ratings and movies datasets on the 'movieId' column

In [10]:
df = pd.merge(ratings,movies,on='movieId')
df= df.drop(['timestamp'], axis=1)
df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller


extract release year from title

In [11]:
def extract_release_year(title):

    try:
        release_year = int(title.split('(')[-1].strip(')'))
    except:
        release_year = None
    return release_year

df['release_year'] = movies['title'].apply(extract_release_year)
movies['release_year'] = movies['title'].apply(extract_release_year)

# Create Figures 🛠


Top-Rated Movies by Release Year

In [12]:
movie_ratings = df.groupby(['movieId', 'title', 'release_year'], as_index=False)['rating'].mean()
top_rated_movies = movie_ratings.sort_values(by=['release_year', 'rating'], ascending=False).groupby('release_year').head(1)

In [13]:
fig_y = px.bar(top_rated_movies, x='release_year', y='rating', hover_data=['title'], labels={'release_year': 'Release Year', 'rating': 'Average Rating'})
fig_y.update_layout(title='Top-Rated Movies by Release Year', xaxis_title='Release Year', yaxis_title='Average Rating',
                    paper_bgcolor='rgba(255, 255, 255, 0.8)',
    plot_bgcolor='rgba(255, 255, 255, 0.8)')

fig_y.update_traces(marker_color='#b8b8ff')
fig_y.show()

Top n Movies by Average Rating

In [14]:
movie_ratings_agg = df.groupby(['movieId', 'title'], as_index=False)['rating'].mean()

In [15]:




def create_movie_popularity_plot(n):
    top_n_movies = movie_ratings_agg.nlargest(n, 'rating')
    fig = px.bar(top_n_movies, x='title', y='rating', labels={'title': 'Movie Title', 'rating': 'Average Rating'})
    fig.update_layout(title=f'Top {n} Movies by Average Rating', xaxis_title='Movie Title', yaxis_title='Average Rating')
    return fig


figs = [create_movie_popularity_plot(i) for i in range(1, 11)]

buttons = [{'label': str(i), 'method': 'update', 'args': [{'x': [figs[i-1].data[0].x], 'y': [figs[i-1].data[0].y]}, {'title': f'Top {i} Movies by Average Rating', 'xaxis': {'title': 'Movie Title'}, 'yaxis': {'title': 'Average Rating'}}]} for i in range(1, 11)]

fig7 = go.FigureWidget(
    data=[go.Bar(x=[], y=[])],
    layout=go.Layout(
        paper_bgcolor='rgba(255, 255, 255, 0.8)',
    plot_bgcolor='rgba(255, 255, 255, 0.8)',
        title={
            'text': 'Top n Movies by Average Rating:',
            'font': {
                'color': 'black'
            }},
        xaxis=dict(range=[0, 10], autorange=False, showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(range=[0, 10], autorange=False, showgrid=False, zeroline=False, showticklabels=False),
        updatemenus=[dict(
            type='dropdown',
            buttons=buttons
        )]
    )
)
fig7.update_traces(marker_color='#99C1DE')

fig7.show()

Explore Movie Ratings by Genre

In [16]:
def filter_movies_by_genre(genre):


   filtered_movies = movies[movies['genres'].str.contains(genre)]


   movie_ratings = ratings.groupby('movieId', as_index=False)['rating'].mean()


   rated_movies = pd.merge(filtered_movies, movie_ratings, on='movieId', how='inner')


   sorted_movies = rated_movies.sort_values(by='rating', ascending=False)

   return sorted_movies
genre_dropdown = widgets.Dropdown(options=movies['genres'].unique(), description='Genre:')
movie_dropdown = widgets.Dropdown(description='Movie:')

def on_dropdown_change(change):
    genre = change.new
    filtered_movies = filter_movies_by_genre(genre)
    movie_options = [{'movie': title} for title, _ in zip(filtered_movies['title'], filtered_movies['movieId'])]
    movie_dropdown.options = movie_options

def on_movie_change(change):
    movie_id = change.new
    movie = movies[movies['movieId'] == movie_id]
    if not movie.empty:
        title = movie.iloc[0]['title']
        genre = movie.iloc[0]['genres']
        filtered_ratings = ratings[ratings['movieId'] == movie_id]
        fig = px.histogram(filtered_ratings, x='rating', nbins=20, labels={'rating': 'Rating'}, opacity=0.7)
        fig.update_layout(title=f'Rating Distribution for "{title}" ({genre})', xaxis_title='Rating', yaxis_title='Count')
        fig.show()

genre_dropdown.observe(on_dropdown_change, names='value')
movie_dropdown.observe(on_movie_change, names='value')


container_layout = widgets.Layout(width='800px', border='none', padding='10px', margin='5px', height='800px')

title = widgets.HTML(value="<h2 style='text-align:center'>Explore Movie Ratings by Genre</h2>")
container = widgets.VBox([title, genre_dropdown, movie_dropdown], layout=container_layout)

display(container)

Top 10 Genres by Number of Movies

In [17]:
genres_count = movies.groupby('genres')['movieId'].count().sort_values(ascending=False)

top10 = genres_count[:10]

In [18]:

fig1 = go.Figure(data=[go.Pie(labels=top10.index, values=top10.values)])


fig1.update_layout(
    paper_bgcolor='rgba(255, 255, 255, 0.8)',
    plot_bgcolor='rgba(255, 255, 255, 0.8)',
    title={
        'text': 'Top 10 Genres by Number of Movies:',
        'y': 0.95,
        'font': {
            'color': 'black'
        }
    },
    margin=dict(l=20, r=20, t=40, b=20),
    legend_title='Genres',
    legend=dict(
        x=1.2,
        y=0.5
    )
)

fig1.update_traces(marker_colors=['#99C1DE', '#BCD4E6', '#D6E2E9', '#F0EFEB', '#DBE7E4','#C5DEDD','#FAD2E1','#FDE2E4','#FFF1E6','#EDDCD2'])
fig1.show()

show Most Frequently Watched Movies

In [19]:
movie_counts = df.groupby(['title', 'genres'])['userId'].count().reset_index()

movie_counts = movie_counts.sort_values(by='userId', ascending=False)


title_regex = r"\((.*?)\)"
movie_counts['year'] = movie_counts['title'].str.extract(title_regex)
movie_counts['title'] = movie_counts['title'].str.replace(title_regex, '').str.strip()

<ipython-input-19-fad667e4634b>:8: FutureWarning:

The default value of regex will change from True to False in a future version.



In [20]:
background_color = 'rgba(255, 255, 255, 0.8)'


fig5 = px.bar(movie_counts.head(20), y='title', x='userId', hover_data=['genres', 'year'],
             labels={'title': 'Movie Title', 'userId': 'Number of Views'})


fig5.update_traces(marker_color='rgba(25, 118, 210, 0.6)')


fig5.update_layout(
    title={
        'text': 'Most Frequently Watched Movies:',
        'font': {'color': 'black'}
    },
    xaxis_title='Number of Views',
    yaxis_title='Movie Title',
    plot_bgcolor=background_color,
    paper_bgcolor='rgba(255,255,255,0.8)'
)
fig5.update_yaxes(title_text="")
fig5.update_traces(marker_color='#bcd4e6')
fig5.show()

In [21]:
df['genres'] = df['genres'].str.split('|')


merged_df = df.explode('genres')
genre_ratings = merged_df.groupby(['genres', 'title','movieId'])['rating'].mean().reset_index()


top_movies = (genre_ratings.groupby('genres')
                            .apply(lambda x: x.nlargest(5, 'rating'))
                            .reset_index(drop=True))
post = pd.merge(links,images,on='imdbId')
top_movies = pd.merge(top_movies,post,on='movieId')
top_movies = top_movies.query('genres != "no genres listed" ')





In [22]:
top_movies

,genres,title,movieId,rating,imdbId,tmdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,(no genres listed),The Adventures of Sherlock Holmes and Doctor W...,147250,5.0,229922,127605.0,http://www.imdb.com/title/tt229922,The Adventures of Sherlock Holmes and Doctor W...,8.6,Crime|Mystery,https://images-na.ssl-images-amazon.com/images...
1,Action,Alien Contamination (1980),6835,5.0,82000,39916.0,http://www.imdb.com/title/tt82000,Contamination (1980),5.3,Horror|Sci-Fi,https://images-na.ssl-images-amazon.com/images...
2,Horror,Alien Contamination (1980),6835,5.0,82000,39916.0,http://www.imdb.com/title/tt82000,Contamination (1980),5.3,Horror|Sci-Fi,https://images-na.ssl-images-amazon.com/images...
3,Sci-Fi,Alien Contamination (1980),6835,5.0,82000,39916.0,http://www.imdb.com/title/tt82000,Contamination (1980),5.3,Horror|Sci-Fi,https://images-na.ssl-images-amazon.com/images...
4,Action,"Assignment, The (1997)",1631,5.0,118647,18355.0,http://www.imdb.com/title/tt118647,The Assignment (1997),6.9,Action|Crime|Thriller,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...,...,...,...
86,War,Come and See (Idi i smotri) (1985),6818,5.0,91251,25237.0,http://www.imdb.com/title/tt91251,Come and See (1985),8.3,Drama|War,https://images-na.ssl-images-amazon.com/images...
87,Western,Duel in the Sun (1946),3792,5.0,38499,32275.0,http://www.imdb.com/title/tt38499,Duel in the Sun (1946),6.9,Drama|Romance|Western,https://images-na.ssl-images-amazon.com/images...
88,Western,Trinity and Sartana Are Coming (1972),128087,5.0,69409,102959.0,http://www.imdb.com/title/tt69409,Trinity and Sartana Are Coming (1972),4.5,Western|Comedy,https://images-na.ssl-images-amazon.com/images...
89,Western,Bone Tomahawk (2015),144976,4.5,2494362,294963.0,http://www.imdb.com/title/tt2494362,Bone Tomahawk (2015),7.1,Adventure|Drama|Horror,https://images-na.ssl-images-amazon.com/images...


In [23]:
'''movie_title = 'Brother (1997)'
movie = top_movies[top_movies['Title'] == movie_title]
poster_url = movie['Poster'].iloc[0]

print(poster_url)'''

"movie_title = 'Brother (1997)'\nmovie = top_movies[top_movies['Title'] == movie_title]\nposter_url = movie['Poster'].iloc[0]\n\nprint(poster_url)"

In [24]:
'''title_to_poster = {
    'Advise and Consent (1962)': 'https://m.media-amazon.com/images/M/MV5BMjE3NzU1MTc0Nl5BMl5BanBnXkFtZTgwMzUxNTIxMDE@._V1_.jpg',
    'You Only Live Once (1937)': 'https://m.media-amazon.com/images/M/MV5BOTYzZTUxN2EtOWVlYy00ZjMwLWI0NGQtMDNmNWRkNDg5MWJmXkEyXkFqcGdeQXVyMjA0MzYwMDY@._V1_.jpg',
    'Brother (Brat) (1997)': 'https://m.media-amazon.com/images/M/MV5BOTMyMmIyYjUtYzZkZS00NTIxLTk4ODItNWI4ZWUzNDA5MWY4XkEyXkFqcGdeQXVyMTAzMDg2MjMx._V1_FMjpg_UX1000_.jpg',
    'Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986)': 'https://www.imfdb.org/images/thumb/0/0d/XX_vek_nachinaetsy_DVD.jpg/300px-XX_vek_nachinaetsy_DVD.jpg',
    'Babes in Toyland (1934)': 'https://upload.wikimedia.org/wikipedia/en/a/a7/L%26H_Babes_in_Toyland_1934.jpg',
    'True Stories (1986)': 'https://m.media-amazon.com/images/M/MV5BYWU0Njc3YTYtNDEyYi00ZWNhLTk5YWQtYzhhNTM5OTYzOTVhXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_.jpg',
    'Journey to the West: Conquering the Demons (Daai wa sai you chi Chui mo chun kei) (2013)': 'https://m.media-amazon.com/images/M/MV5BMTQzMzUxNzk0NV5BMl5BanBnXkFtZTgwMTEwOTE4MDE@._V1_.jpg',
    'Boy Eats Girl (2005)': 'https://m.media-amazon.com/images/M/MV5BYjU4ZDgyZDgtOTgyNy00M2IxLTgwZjctOWFiZjllYzcyYzM1XkEyXkFqcGdeQXVyMTY5Nzc4MDY@._V1_.jpg',
    'Come and See (Idi i smotri) (1985)': 'https://m.media-amazon.com/images/M/MV5BODM4Njg0NTAtYjI5Ny00ZjAxLTkwNmItZTMxMWU5M2U3M2RjXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_.jpg',
    'Long Goodbye, The (1973)': 'https://m.media-amazon.com/images/M/MV5BNGZkZjc2ODEtOGY2ZC00NTAxLTkwNmMtNjQyOGFlZTNiNjE4XkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_FMjpg_UX1000_.jpg'
}
for title, poster in title_to_poster.items():
    top_movies.loc[top_movies['title'] == title, 'poster'] = poster'''

"title_to_poster = {\n    'Advise and Consent (1962)': 'https://m.media-amazon.com/images/M/MV5BMjE3NzU1MTc0Nl5BMl5BanBnXkFtZTgwMzUxNTIxMDE@._V1_.jpg',\n    'You Only Live Once (1937)': 'https://m.media-amazon.com/images/M/MV5BOTYzZTUxN2EtOWVlYy00ZjMwLWI0NGQtMDNmNWRkNDg5MWJmXkEyXkFqcGdeQXVyMjA0MzYwMDY@._V1_.jpg',\n    'Brother (Brat) (1997)': 'https://m.media-amazon.com/images/M/MV5BOTMyMmIyYjUtYzZkZS00NTIxLTk4ODItNWI4ZWUzNDA5MWY4XkEyXkFqcGdeQXVyMTAzMDg2MjMx._V1_FMjpg_UX1000_.jpg',\n    'Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986)': 'https://www.imfdb.org/images/thumb/0/0d/XX_vek_nachinaetsy_DVD.jpg/300px-XX_vek_nachinaetsy_DVD.jpg',\n    'Babes in Toyland (1934)': 'https://upload.wikimedia.org/wikipedia/en/a/a7/L%26H_Babes_in_Toyland_1934.jpg',\n    'True Stories (1986)': 'https://m.media-amazon.com/images/M/MV5BYWU0Njc3YTYtNDEyYi00ZWNhLTk5YWQtYzhhNTM5OTYzOTVhXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_.jpg',\n    'Journey to the West: Conquering the Dem

In [25]:
fig_tree = px.treemap(top_movies, path=['genres', 'title'], values='rating',custom_data=["Poster","title"],
                 color='rating', color_continuous_scale='Magma',
                 labels={'rating': 'Average Rating'})

fig_tree.update_layout(paper_bgcolor='rgba(255, 255, 255, 0.8)',
    plot_bgcolor='rgba(255, 255, 255, 0.8)',
                   title="Top 5 Movies for Each Category",

                  clickmode='event+select'
                  )

fig_tree.show()

In [26]:
df2 = pd.merge(links, images, on='imdbId')

In [27]:
rating_matrix = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
rating_matrix_sparse = csr_matrix(rating_matrix.values)

# Knn Item Recommender

In [28]:
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
model.fit(rating_matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [29]:
def knn_item_recommender(movie_title, no_of_recommendations, model):
    id_of_movie = process.extractOne(movie_title, movies['title'])[2]
    distances, indices = model.kneighbors(rating_matrix_sparse[id_of_movie], n_neighbors=no_of_recommendations+1)
    indices = np.delete(indices, 0)
    recommendations = movies.iloc[indices]['title'].tolist()
    return recommendations

recommendations = knn_item_recommender('Jumanji (1995)', 5, model)
print(recommendations)

'''for title in recommendations:
    movie = df2.loc[df2['Title'] == title]
    if len(movie) == 0:
        print(f"No image available for {title}")
        continue

    url = movie.iloc[0]['Poster']
    if pd.isna(url):
        print(f"No image available for {title}")
        continue

    response = requests.get(url)
    try:
        img = Image.open(BytesIO(response.content))
        img.show()
    except UnidentifiedImageError as e:
        print(f"Error opening image for {title}: {e}")'''

['Lion King, The (1994)', 'Mrs. Doubtfire (1993)', 'Mask, The (1994)', 'Jurassic Park (1993)', 'Home Alone (1990)']


'for title in recommendations:\n    movie = df2.loc[df2[\'Title\'] == title]\n    if len(movie) == 0:\n        print(f"No image available for {title}")\n        continue\n\n    url = movie.iloc[0][\'Poster\']\n    if pd.isna(url):\n        print(f"No image available for {title}")\n        continue\n\n    response = requests.get(url)\n    try:\n        img = Image.open(BytesIO(response.content))\n        img.show()\n    except UnidentifiedImageError as e:\n        print(f"Error opening image for {title}: {e}")'

In [30]:
nan_mask = pd.isna(df2['Poster'])
nan_titles = df2.loc[nan_mask, 'Title']
if len(nan_titles) > 0:
    print("The following movies have a NaN value in the 'Poster' column:")
    for title in nan_titles:
        print(title)
else:
    print("There are no NaN values in the 'Poster' column")

The following movies have a NaN value in the 'Poster' column:
Nothing to Lose (1994)
Loser (1991)
Voices from the List (2004)
Ready Player One (2018)


In [31]:
movies['genres'] = movies['genres'].str.split('|')
movies = movies.explode('genres')
num_users = len(ratings['userId'].unique())
num_genres = len(movies['genres'].unique())
num_movies = len(movies['movieId'].unique())

In [32]:
'''import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/sample_data')'''

"import os\nfrom google.colab import drive\ndrive.mount('/content/drive')\nos.chdir('/content/sample_data')"

In [33]:
genres = sorted(list(set('|'.join(movies['genres']).split('|'))))

In [36]:
dafault = '/content/sample_data/gray-movie-written-icon-png-UpaYYD.png'
test_base64 = base64.b64encode(open(dafault, 'rb').read()).decode('ascii')
default_image = '/content/sample_data/gray-movie-written-icon-png-UpaYYD.png'
default_base64 = base64.b64encode(open(default_image, 'rb').read()).decode('ascii')

In [37]:
# Titles that should show the default image
default_titles = ['Advise and Consent (1962)', 'You Only Live Once (1937)', 'Brother (Brat) (1997)',
                  'Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986)',
                  'Babes in Toyland (1934)', 'True Stories (1986)',
                  'Journey to the West: Conquering the Demons (Daai wa sai you chi Chui mo chun kei) (2013)',
                  'Boy Eats Girl (2005)', 'Come and See (Idi i smotri) (1985)', 'Long Goodbye, The (1973)']

In [365]:
card = dbc.Card(
    dbc.CardBody([
        html.Label('Select a genre:', style={'fontSize': '20px'}),
        dcc.Dropdown(
            id='genre-dropdown',
            options=[{'label': genre, 'value': genre} for genre in genres],
            value='Action',
            style={'fontSize': '20px','color':"##bcd4e6"}
        ),
        html.Label('Select a movie:', style={'fontSize': '20px'}),
        dcc.Dropdown(
            id='movie-dropdown',
            style={'fontSize': '20px','color':"##bcd4e6"}
        ),
    ])
)

top_10_recommendations

In [400]:
%run /content/sample_data/model/predict.py

In [445]:
predictions_df, best_movies_df = top_5_recommendations(5)

# Define the URL of the default movie image
default_image_url = "https://icon-library.com/images/movie-icon-ico/movie-icon-ico-4.jpg"

# Create a list of movie cards
movie_cards = []
for index, row in best_movies_df.iterrows():
    card = dbc.Card(
        [
            dbc.CardImg(src=default_image_url, top=True, style={"height": "200px", "object-fit": "cover"}),
            dbc.CardBody(
                [
                    html.H5(row['title'], className="card-title", style={"font-weight": "bold",'color': 'white'}),
                ]
            )
        ],
        style={
            "display": "flex",
            "flex-direction": "column",
            "justify-content": "space-between",
            "height": "90%",
            "margin-bottom": "20px",
            "border": "1px solid #ced4da",
            "box-shadow": "0 4px 8px 0 rgba(0,0,0,0.2)",
            "transition": "0.3s",
            "border-radius": "5px",
            "padding": "10px",
            "margin-right": "10px",
            "margin-left": "10px"
        }
    )
    movie_cards.append(dbc.Col(card, width=4))


5/5 [==============================] - 8s 28ms/step


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.8s finished



#Main code for dashboard

In [446]:
app1=JupyterDash(__name__)
app1.layout = html.Div(style={'background-image': 'url("https://repository-images.githubusercontent.com/275336521/20d38e00-6634-11eb-9d1f-6a5232d0f84f")',
                              'background-size': 'cover',
                              'height': '50%','width':'100%',
                              'background-color': 'rgba(76, 76, 76, 0.0)'},
                      children=[
    html.Div([
        html.H1(html.B('🎬 The Movie Maze: A Dashboard for Analyzing Movie Data'),
                style={'color': 'white', 'fontSize': 35, 'textAlign': 'center',
                       'fontFamily': 'Courier New, monospace', 'margin': '20px','background-color': 'rgba(0, 0, 0, 0.5)'}),
        html.Hr(style={'width': '10', 'textAlign': 'center'}),

        html.Div([
            html.Div([
                html.H2('Movies 📽', style={'textAlign': 'center'}),
                html.H3(num_movies, style={'textAlign': 'center'})
            ], className='card', style={'color': 'white','display': 'inline-block', 'border': '1px solid #ddd', 'width': '30%', 'marginRight': '2%'}),
            html.Div([
                html.H2('People 🧑', style={'textAlign': 'center'}),
                html.H3(num_users, style={'textAlign': 'center'})
            ], className='card', style={'color': 'white','display': 'inline-block', 'border': '1px solid #ddd', 'width': '30%', 'marginRight': '2%'}),
            html.Div([
                html.H2('Genres 🤔', style={'textAlign': 'center'}),
                html.H3(num_genres, style={'textAlign': 'center'})
            ], className='card', style={'color': 'white','display': 'inline-block', 'border': '1px solid #ddd', 'width': '30%'})
        ], className='card-container', style={'textAlign': 'center'}),
        html.Hr(style={'width': '10', 'textAlign': 'center'}),


         html.Div([
        html.Div([
            html.H1('Reel Recommendations', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center'}),
        dcc.Input(id='input', placeholder='Enter movie title', type='text', value='Bloodmoon (1997)',
              style={'width': '200px', 'height': '33px', 'fontSize': '18px', 'marginLeft': '2%'}),
        html.Button('Recommend', id='button', style={'margin': '10px', 'width': '200px', 'height': '40px', 'fontSize': '18px', 'background-color': '#bcd4e6','color':'white'}),
        html.Div(id='output', style={'display': 'flex', 'flex-wrap': 'wrap'})
    ],
    style={'color': 'white', 'marginLeft': '0.8%'}),

        html.Hr(style={'width':'10','textAlign':'center'}),

        html.Div([
            html.Div([
            html.H1('Top 5 Movie Recommendations for You', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                 'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center'}),

        html.Br(),
        dbc.Row(movie_cards, style={"display": "flex", "flex-wrap": "nowrap", "overflow-x": "auto", "justify-content": "center"}),
    ],
    style={"max-width": "1500px",'margin-right': '2%','margin-left': '2%'}),
        html.Hr(style={'width':'10','textAlign':'center'}),
        html.Div([
            html.Div([
            html.H1('Movie Insights: Genres and Viewer Trends', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center', 'marginBottom': '30px'}),
            html.Div([
                dcc.Graph(
                    id='Top 10 Genres by Number of Movies',
                    figure=fig1
                )
            ], style={'width': '37%', 'margin': 'auto', 'float': 'left', 'marginLeft': '2.6%'}),

            # Add the second graph
            html.Div([
                dcc.Graph(
                    id='most-frequently-watched-movies',
                    figure=fig5
                )
            ], style={'width': '53%', 'margin': 'auto', 'float': 'left', 'marginRight': '2.6%'})
        ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'flexWrap': 'wrap',
                  'marginTop': '20px'}),
        html.Div([
            html.H1('Top Rated Movies', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center', 'marginBottom': '40px', 'marginTop': '40px'}),
        html.Div([
            dcc.Graph(
                id='Top n Movies by Average Rating',
                figure=fig7,
                style={'marginTop': '7px', 'marginRight': '2.6%', 'marginLeft': '2.6%'}
            )
        ]),
        html.Div([
            html.H1('Discovering Movie Ratings and Genres', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center', 'marginBottom': '40px', 'marginTop': '40px'}),

        html.Div([



    html.Div([
    html.Label('Select a genre:', style={'fontSize': '20px'}),
    dcc.Dropdown(
        id='genre-dropdown',
        options=[{'label': genre, 'value': genre} for genre in genres],
        value='Animation',
        style={'fontSize': '20px'}
    )
], style={'width': '30%', 'display': 'inline-block', 'marginTop': '10px', 'verticalAlign': 'middle'}),

html.Div([
    html.Label('Select a movie:', style={'fontSize': '20px'}),
    dcc.Dropdown(
        id='movie-dropdown',
        style={'fontSize': '20px'}
    )
], style={'width': '30%', 'display': 'inline-block', 'float': 'right', 'marginTop': '10px', 'verticalAlign': 'middle'})

], style={'backgroundColor': 'rgba(255, 255, 255, 0.8)',
          'padding': '10px', 'marginTop': '10px', 'marginBottom': '30px',
          'backgroundSize': 'cover', 'width': '1247px',
          'height': '350px', 'marginRight': '2.5%', 'marginLeft': '2.5%'}),
        html.Div([
            html.H1('Explore Top Rated Movies by Genre with Posters', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center', 'marginBottom': '40px', 'marginTop': '40px'}),
      html.Div([
    html.Div([
        dcc.Graph(id='treemap', figure=fig_tree, style={'marginRight': '2.5%', 'marginLeft': '2.5%', 'width': '40%', 'display': 'inline-block',
                                                        'height': '500px'}),
        html.Div(id='card', style={'marginLeft': '2.5%', 'display': 'inline-block', 'width': '40%', 'height': '500px'})
    ])
], style={'marginTop': '10px'}),
        html.Div([
            html.H1('Exploring the Top-Rated Movies by Release Year', style={'margin-right': '2%', 'color': 'white','margin-left': '2%'})
        ], style={'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px','margin-right': '2%','margin-left': '2%',
                  'height': '20px', 'width': '1220px', 'display': 'flex', 'align-items': 'center', 'marginBottom': '40px', 'marginTop': '40px'}),
        html.Div([
            dcc.Graph(
                id='Top-Rated Movies by Release Year',
                figure=fig_y,
                style={'marginTop': '7px', 'marginRight': '2.6%', 'marginLeft': '2.6%','marginBottom': '10px'}
            ),
            html.Br()
        ])
    ])
])




# callbacks

In [447]:
@app1.callback(
    dash.dependencies.Output('output', 'children'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('input', 'value')]
)
def update_output(n_clicks, value):
    if n_clicks is None:
        value = 'Love Affair (1994)'
    recommendations = knn_item_recommender(value, 5, model)
    cards = []
    for title in recommendations:
        movie = df2.loc[df2['Title'] == title]
        if len(movie) == 0:
            url = 'https://www.transparentpng.com/thumb/movie/gray-movie-written-icon-png-UpaYYD.png'
        else:
            url = movie.iloc[0]['Poster']
            if pd.isna(url):
                url = 'https://www.transparentpng.com/thumb/movie/gray-movie-written-icon-png-UpaYYD.png'
            else:
                try:
                    response = requests.get(url)
                    response.raise_for_status()
                    img = Image.open(BytesIO(response.content))
                except (requests.exceptions.HTTPError, requests.exceptions.RequestException, UnidentifiedImageError):
                    url = 'https://www.transparentpng.com/thumb/movie/gray-movie-written-icon-png-UpaYYD.png'

        cards.append(html.Div([
            html.Img(src=url, width=220, style={'margin': '10px'}),
            html.P(title)
        ], style={'margin-left': '10px','margin-right': '10px', 'margin-bottom': '10px'}))
    return cards



@app1.callback(
    Output('movie-dropdown', 'options'),
    [Input('genre-dropdown', 'value')]
)
def update_movie_options(selected_genre):

    filtered_movies = filter_movies_by_genre(selected_genre)

    movie_options = [{'label': title, 'value': movie_id} for title, movie_id in zip(filtered_movies['title'], filtered_movies['movieId'])]
    return movie_options

@app1.callback(
    Output('movie-dropdown', 'value'),
    [Input('movie-dropdown', 'options')]
)


def update_selected_movie(options):
    return options[0]['value']

@app1.callback(Output('card', 'children'), [Input('treemap', 'clickData')])
def update_card(clickData):
    if clickData:
        poster_url = clickData['points'][0]['customdata'][0]
        title = clickData['points'][0]['customdata'][1]


        if title in default_titles:
            return html.Div([
                html.Div(style={'background-image': 'url("tree.jpg")', 'backgroundColor': 'rgba(255, 255, 255, 0.8)'
                    },
                         children=[
                             html.Img(src='data:image/png;base64,{}'.format(default_base64), width=200, height=200),
                             html.H4(title) if title else ''
                         ])
            ])


        if poster_url and validators.url(poster_url):
            try:

                img_base64 = base64.b64encode(requests.get(poster_url).content).decode('ascii')
                return html.Div([
                    html.Div(style={'background-image': 'url("tree.jpg")', 'background-size': 'cover',
                                     'border-radius': '10px',
                                     'backgroundColor': 'rgba(255, 255, 255, 0.8)'},
                             children=[
                                 html.Img(src='data:image/png;base64,{}'.format(img_base64), width=200),
                                 html.H4(title) if title else ''
                             ])
                ])
            except:

                pass

        # Use the default image if the poster URL is invalid or the image cannot be loaded
        return html.Div([
            html.Div(style={'background-image': 'url("tree.jpg")', 'backgroundColor': 'rgba(255, 255, 255, 0.8)'
                             },
                     children=[
                         html.Img(src='data:image/png;base64,{}'.format(default_base64), width=400, height=400),
                         html.H4(title) if title else ''
                     ])
        ])
    else:

        return html.Div([
            html.Div(style={'background-image': 'url("tree.jpg")'},
                     children=[
                         html.Img(src='data:image/png;base64,{}'.format(default_base64), width=450, height=500),
                         html.H4('')
                     ])
        ])

In [448]:

if __name__ == '__main__':
    app1.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>